# Recon2.2plusthreeCompetitionII.json [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
from utils import findBiomarkers

In [2]:
M1 = cobra.io.load_json_model('Recon2.2plusthreeIndependent(previous).json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
M=M1.copy()

In [4]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_glc_e_INB.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_h2o_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
# deleting liver and brain secreted reactions which no metabolites could not secrete in liver and brain.
M.reactions.EX_acac_e.upper_bound=0
M.reactions.EX_bhb_e.upper_bound=0
M.reactions.EX_co2_e.upper_bound=0
M.reactions.EX_h_e.upper_bound=0
M.reactions.EX_h2o_e.upper_bound=0
M.reactions.EX_lac_D_e.upper_bound=0
M.reactions.EX_lac_L_e.upper_bound=0
M.reactions.EX_nh4_e.upper_bound=0
M.reactions.EX_pi_e.upper_bound=0
M.reactions.EX_so4_e.upper_bound=0
M.reactions.EX_urea_e.upper_bound=0
M.reactions.EX_phpyr_e.upper_bound=0
M.reactions.EX_acac_e_INB.upper_bound=0
M.reactions.EX_bhb_e_INB.upper_bound=0
M.reactions.EX_co2_e_INB.upper_bound=0
M.reactions.EX_h_e_INB.upper_bound=0
M.reactions.EX_h2o_e_INB.upper_bound=0
M.reactions.EX_lac_D_e_INB.upper_bound=0
M.reactions.EX_lac_L_e_INB.upper_bound=0
M.reactions.EX_nh4_e_INB.upper_bound=0
M.reactions.EX_pi_e_INB.upper_bound=0
M.reactions.EX_so4_e_INB.upper_bound=0
M.reactions.EX_urea_e_INB.upper_bound=0
M.reactions.EX_phpyr_e_INB.upper_bound=0

In [5]:
M.reactions.EX_TH_B_INB.upper_bound=0

In [6]:
### Adding the transport reaction for metabolites from liver to blood and from brain to blood
### We assumed that they did nou use any transporters.
MetaAddTR=['acac','bhb','co2','h','h2o','lac_D','lac_L','nh4','pi','so4','urea','dopa','adrnl','srtn']

In [7]:
for META in MetaAddTR:
    META_c=META + '_c'
    META_e_INB=META + '_e'+'_INB'
    from cobra import Model, Reaction, Metabolite
    # Adding reaction from liver to blood
    rxn_id = f'{META}_TOBD'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {META} from liver to blood'
    reaction.lower_bound = -1000  
    reaction.upper_bound = 1000

    # Create metabolite object
    meta = Metabolite(f'{META}_BD', formula='', name='', compartment='blood')
    meta2=M.metabolites.get_by_id(META_c)
    reaction.add_metabolites({meta: 1.0, meta2:-1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])
    
    # Adding reaction from blood to brain
    rxn_id = f'{META}_TOB'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {META} from blood to brain'
    reaction.lower_bound = -1000  
    reaction.upper_bound = 1000

    # Create metabolite object
    meta = Metabolite(f'{META}_BD', formula='', name='', compartment='blood')
    meta3=M.metabolites.get_by_id(META_e_INB)
    reaction.add_metabolites({meta: -1.0, meta3:1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])

In [8]:
BloodMeta1=[meta for meta in M.metabolites if '_BD' in meta.id
            and '34dhphe' not in meta.id 
            and 'phpyr_BD'not in meta.id and 'To' not in meta.id]
len(BloodMeta1)

36

In [9]:
### Add some exchange reactions (all together) in the blood
for META in BloodMeta1:
    meta_id_c = META.id
    from cobra import Model, Reaction, Metabolite
    # Adding exchange reaction
    rxn_id = f'EX_{meta_id_c}'
    reaction = Reaction(rxn_id)
    reaction.name = f'Exchange of {meta_id_c}'
    reaction.lower_bound = 0  
    reaction.upper_bound = 1000
    # Create metabolite object
    meta=M.metabolites.get_by_id(META.id)
    reaction.add_metabolites({meta: -1.0})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])

In [10]:
## O2 uptake from blood and not from liver, the rest medium metabolites take up from liver
M.reactions.EX_o2_e.lower_bound=0
M.reactions.EX_o2_BD.lower_bound=-1000

In [11]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

0.16888753778858667

In [12]:
SecretedRXN=[rxn for rxn in M.reactions if M.reactions.get_by_id(rxn.id).upper_bound>0 and 'EX_' in rxn.id]
len(SecretedRXN)

38

In [13]:
cobra.io.save_json_model(M,'Recon2.2plusthreeIndependent.json')